# **EXPLORATORY DATA ANALYSIS (EDA)**

Importamos librerías para el trabajo

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
diabetes_df = pd.read_csv('../data/raw/diabetes.csv')

In [ ]:
diabetes_df.head()

In [ ]:
print("Número de filas y columnas presentes en el dataframe: ", diabetes_df.shape)

In [ ]:
diabetes_df.info()

In [ ]:
diabetes_df.describe().T

In [ ]:
sns.set_theme(style="whitegrid")

# Crear el gráfico countplot
plt.figure(figsize=(6, 4))
sns.countplot(x='Outcome', data=diabetes_df)

plt.xlabel('Padecimiento de diabétes')
plt.show()

In [ ]:
sns.set_theme(style="whitegrid")

# Crear el gráfico countplot
plt.figure(figsize=(6, 4))
ax = sns.countplot(x='Outcome', data=diabetes_df)

# Agregar el valor y el porcentaje en paréntesis en cada barra
total = len(diabetes_df)
for p in ax.patches:
    height = p.get_height()
    percentage = (height / total) * 100
    ax.annotate(f'{height} ({percentage:.1f}%)', (p.get_x() + p.get_width() / 2., height),
                ha='center', va='bottom', fontsize=12)

plt.xlabel('Padecimiento de diabétes')
plt.show()

In [ ]:
diabetes_df['Outcome'].value_counts()

Creamos el array de características (features) y target

In [ ]:
X = diabetes_df.drop('Glucose', axis=1).values
y = diabetes_df['Glucose'].values

print(y.shape, X.shape)

#### **Valores nulos**

In [ ]:

# ! NO MODIFICAR
# * Función creada para contar el porcentaje de missings existente por cada variable

def msv_1(data, thresh = 20, color = 'black', edgecolor = 'black', height = 3, width = 15):
    
    plt.figure(figsize = (width, height))
    percentage = (data.isnull().mean()) * 100
    percentage.sort_values(ascending = False).plot.bar(color = color, edgecolor = edgecolor)
    plt.axhline(y = thresh, color = 'r', linestyle = '-')
    
    plt.title('Porcentaje de missing values por columna', fontsize=20, weight='bold' , y=1.1)
    
    plt.text(len(data.isnull().sum()/len(data))/1.7, thresh+2.5, f'Columns con más de {thresh}% missing values', fontsize=12, color='crimson',
         ha='left' ,va='top')
    plt.text(len(data.isnull().sum()/len(data))/1.7, thresh - 0.5, f'Columnas con menos de {thresh}% missing values', fontsize=12, color='green',
         ha='left' ,va='top')
    plt.xlabel('Columnas', size=15, weight='bold')
    plt.ylabel('Porcentaje de missing values')
    plt.yticks(weight ='bold')
    
    return plt.show()

In [ ]:
msv_1(diabetes_df, 20, color=sns.color_palette('Reds',15))

Pero en esta base de datos en particular nos podemos cuestionar lo siguiente: ¿Es posible tener valores cero para indicadores como glucosa? Suponiendo que la respuesta es que no, los valores cero deberíamos de considerarlo como Missing Data y suprimirlos de nuestro dataset.

In [ ]:
# Reemplazamos con missings aquellos valores cero para indicacdores de salud
diabetes_df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']] = diabetes_df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']].replace(0, np.nan)

In [ ]:
msv_1(diabetes_df, 20, color=sns.color_palette('Reds',15))

Ahora observamos que varias de nuestras características (features) tenían valores cero. Vamos a considerar entonces estos valores cero como missing data y suprimirlos de nuestro dataset.

#### **Exploratory Data Analysis (EDA)**

En esta sección, haremos un análisis de datos exploratorio básico para tener una "sensación" de los datos, verificaremos las distribuciones, las correlaciones, etc. de las diferentes columnas e intentaremos eliminar los valores nulos presentes.

In [ ]:
# Crear una figura con 4 filas y 2 columnas de subgráficos, con un tamaño de 15x10 pulgadas
fig, axes = plt.subplots(4, 2, figsize=(15, 10))
# Aplanar la matriz de subgráficos para facilitar el acceso
axes = axes.flatten()
# Inicializar un índice para recorrer las columnas del DataFrame
ax_idx = 0
# Obtener las columnas del DataFrame, excluyendo la columna 'Outcome'
columns = diabetes_df.drop('Outcome', axis=1).columns
# Iterar a través de las columnas y crear histogramas en cada subgráfico
for col in columns:
    # Crear un histograma para la columna actual en el subgráfico correspondiente
    diabetes_df[col].plot(kind='hist', ax=axes[ax_idx], title=col)
    # Incrementar el índice para pasar al siguiente subgráfico
    ax_idx += 1
# Agregar un título general para los subgráficos
plt.suptitle('Histograma de feature variables')
# Ajustar automáticamente la disposición de los subgráficos para evitar superposiciones
plt.tight_layout()
# Mostrar los subgráficos
plt.show()

Vamos a tomar un criterio en específico para llenar los valores missings. Si nuestra distribución se encuentra muy sesgada hacia un lado usaremos la mediana, mientras que si tiene una distribución más normal usaremos la media.

Comprobemos la asimetría (Skewness ) de cada una de las columnas. La asimetría se refiere a la cantidad de asimetría en una característica dada o, en otras palabras, a la cantidad de distorsiones de la distribución normal. El pico del histograma representa la moda.

In [ ]:
from scipy.stats import skew

for col in diabetes_df.drop('Outcome', axis = 1).columns:
    print("Skewness for the column {} is {}".format(col, diabetes_df[col].skew()))

Completamos los valores missings:

In [ ]:
diabetes_df['Insulin'] = diabetes_df['Insulin'].fillna(diabetes_df['Insulin'].median()) # Completamos valores nulos con la mediana

for col in ['Glucose', 'BloodPressure', 'SkinThickness', 'BMI']:
    diabetes_df[col] = diabetes_df[col].fillna(diabetes_df[col].mean())

In [ ]:
diabetes_df

In [ ]:
diabetes_df.to_excel("../data/final/diabetes_final.xlsx")

Visualizamos cómo nos queda el gráfico de reporte luego del ajuste:

In [ ]:
msv_1(diabetes_df, 20, color=sns.color_palette('Greens',15))

In [ ]:
diabetes_df.isnull().sum()

Todos los valores nulos fueron tratados ahora.

In [ ]:
def mean_target(var):
    """
    Una función que devolverá los valores medios para la columna 'var' dependiendo de si la persona
    es diabético o no
    """
    return pd.DataFrame(diabetes_df.groupby('Outcome').mean()[var])

In [ ]:
def distplot(col_name):
    """
    Una función que trazará la distribución de la columna 'col_name' para personas diabéticas y no diabéticas por separado.
    """
    # Crear una nueva figura para el gráfico de distribución.
    plt.figure() 
    # Trazar la distribución de 'col_name' para personas con diabetes en rojo.
    ax = sns.distplot(diabetes_df[col_name][diabetes_df.Outcome == 1], color="red", rug=True) 
    # Trazar la distribución de 'col_name' para personas sin diabetes en azul claro.
    sns.distplot(diabetes_df[col_name][diabetes_df.Outcome == 0], color="pink", rug=True)   
    # Agregar una leyenda para indicar las categorías en el gráfico.
    plt.legend(['Diabetes', 'No Diabetes'])

**Embarazos (Pregnancies)**

In [ ]:
distplot('Pregnancies')

In [ ]:
mean_target('Pregnancies')

Podemos observar que el número de embarazos es elevado para las personas diabéticas.

**Insulina (Insulin)**

In [ ]:
distplot('Insulin')

In [ ]:
mean_target('Insulin')

Las personas diabéticas tienden a tener más niveles de insulina.

**Presión Arterial (BloodPressure)**

In [ ]:
distplot('BloodPressure')

In [ ]:
mean_target('BloodPressure')

La media de la presión arterial es mayor en las personas diabéticas que en las personas no diabéticas.

**Glucosa (Glucose)**

In [ ]:
distplot('Glucose')

In [ ]:
mean_target('Glucose')

Las personas diabéticas tienden a tener niveles de glucosa mucho más altos.

#### **Boxplots y su análisis**

Los boxplots (también conocidos como diagramas de caja y bigotes) son una herramienta gráfica utilizada en estadísticas y análisis de datos para visualizar la distribución y resumir estadísticas clave de un conjunto de datos numéricos. Sirven para proporcionar una comprensión rápida y efectiva de la distribución de los datos y para identificar posibles valores atípicos (outliers). Aquí están las principales utilidades de los boxplots:

In [ ]:
sns.boxplot(x = 'Outcome', y = 'Age', data = diabetes_df)
plt.title('Edad vs Outcome')
plt.show()

Podemos observar en este caso que la mediana de las personas con diabétes es mayor que la de las personas sin diabétes. ¿Nos hace sentido?

In [ ]:
sns.boxplot(x = 'Outcome', y = 'BloodPressure', data = diabetes_df, palette = 'Blues')
plt.title('Presión arterial vs Outcome')
plt.show()

La mediana de la presión arterial de las personas diabéticas se sitúa cerca del percentil 75 de las personas no diabéticas. E
n promedio, las personas con diabetes tienden a tener niveles de presión arterial más altos que el 75% de las personas sin diabetes 

In [ ]:
sns.jointplot(x='Age',y='BloodPressure', data=diabetes_df, kind = 'reg', color = 'green')

A medida que aumenta la edad, generalmente la presión arterial también aumenta.

In [ ]:
my_pal = {0: "lightgreen", 1: "lightblue"}
sns.boxplot(x = 'Outcome', y = 'DiabetesPedigreeFunction', data = diabetes_df, palette = my_pal)
plt.title('DPF vs Outcome')
plt.show()

Una gran proporción de personas que tienen un DPF alto no acaban teniendo diabetes. Pero normalmente las personas diabéticas tienen un valor de DPF cercano a 0,5 (percentil 50).

In [ ]:
my_pal = {0: "lightgrey", 1: "lightyellow"}
sns.boxplot(x = 'Outcome', y = 'Glucose', data = diabetes_df, palette = my_pal)
plt.title('Glucosa vs Outcome')
plt.show()

La mediana del nivel de glucosa de las personas diabéticas es mayor que el percentil 75 del nivel de glucosa de las personas no diabéticas. Por lo tanto, tener un nivel alto de glucosa sí aumenta las posibilidades de tener diabetes.

In [ ]:
sns.jointplot(x='Insulin',y='Glucose', data=diabetes_df, kind = 'reg', color = 'red')
plt.show()

Podemos ver que a medida que aumenta el nivel de insulina, también aumenta el nivel de Glucosa.

In [ ]:
sns.boxplot(x = 'Outcome', y = 'Insulin', data = diabetes_df)
plt.title('Insulina vs Outcome')
plt.show()

In [ ]:
my_pal = {0: "lightyellow", 1: "lightpink"}
sns.boxplot(x = 'Outcome', y = 'BMI', data = diabetes_df, palette = my_pal)
plt.title('IMC vs Outcome')
plt.show()

El IMC medio de las personas diabéticas es mayor que el IMC medio de las personas no diabéticas.

#### **Matriz de correlación**

In [ ]:
# Calcula la matriz de correlación entre las columnas del DataFrame.
corr = diabetes_df.corr()
# Crea una máscara triangular superior en la matriz de correlación para ocultar los valores duplicados.
mask = np.triu(np.ones_like(corr, dtype=bool))
# Crea una figura para el gráfico de calor de la matriz de correlación.
f, ax = plt.subplots(figsize=(11, 9))
# Define una paleta de colores divergentes para el gráfico de calor.
cmap = sns.diverging_palette(230, 20, as_cmap=True)
# Trazar un mapa de calor de la matriz de correlación.
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=1.0, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True)

En el mapa de calor anterior, podemos observar que todas las características están débilmente correlacionadas, por lo que elimina la multicolinealidad de la ecuación. La multicolinealidad (también colinealidad) es un fenómeno en el que una variable predictiva en un modelo de regresión múltiple se puede predecir linealmente a partir de las demás con un grado sustancial de precisión. Los modelos como la regresión logística suponen la presencia de no colinealidad entre las características; si hay multicolinealidad, puede provocar un mal rendimiento de dichos modelos.